# Carmen's Code
## Including trajectories, bbpa noise and tp noise and b-spline on Marcel's Polygon

A quick guide to understanding this code:

The first chunk is all the imports needed
some rely on a different type of python and this is commented

Then we give the helper functions for the noise and paths, all of these need to be loaded before the simulator can be used. The plots do not need to be run and will take a while to run so probs best to not bother.

The simulator is at the bottom

In [1]:
# all the necessary imports!

import numpy as np
from numpy.linalg import inv
import math 
import pandas as pd

import IPython
%matplotlib qt
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from bokeh.io import curdoc
from bokeh.layouts import row, widgetbox, gridplot
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Slider, Button
from bokeh.plotting import figure, show

from scipy import interpolate
from scipy.interpolate import BSpline

# to do this bit i needed to do 'pip install shapely' into the command line
# on Windows this required entering 'conda install shapely' in a notebook cell
from shapely.geometry import Point, Polygon # to define polygon object and find closest border point
from shapely.ops import nearest_points # to find closest border point for ports
import matplotlib.path as mpltPath # to check if the point is on the border

import random



## Noise helpers

In [2]:
def e_k(x):
    """legendre functions"""
    output = np.array([x,0.5*(3*x**2-1),0.5*(5*x**3-3*x),0.125*(35*x**4-30*x**2+3),0.125*(63*x**5-70*x**3+15*x)])
    return(output)
    
    

    
def dot(x,I_k,trad_len):
    """ dot product helper for the approximation"""
    n = np.shape(x)[0]
    transform_x = 2/trad_len * x - np.ones(n)
    e_k_x = e_k(transform_x)
    return(np.dot(I_k,e_k_x))




def renorm(col,noise_0,noise_1,trad_len):
    """ Here col is a column of the noise and the time, noise_0 
    is the noise at time 0 and noise_1 is the noise at time 1 
    before renormlisation"""
    col[0] = col[0] - noise_0 -  (col[1]) * (noise_1-noise_0)/trad_len
    return(col)


def renorm_ep(col,noise_0,noise_1,trad_len,end_point):
    """ Here col is a column of the noise and the time, noise_0 
    is the noise at time 0 and noise_1 is the noise at time 1 
    before renormlisation"""
    extra = (end_point/trad_len)*col[1]
    col[0] = col[0] - noise_0 -  (col[1]) * (noise_1-noise_0)/trad_len + extra
    # (end_point/trad_len) * col[1]
    return(col)


def renorm_sp(col,noise_0,noise_1,trad_len,start_point):
    """ Here col is a column of the noise and the time, noise_0 
    is the noise at time 0 and noise_1 is the noise at time 1 
    before renormlisation"""
    
    #the extra bit to allow for the start point
    extra = (-start_point/trad_len)*col[1] + start_point 
    col[0] = col[0] - noise_0 -  (col[1]) * (noise_1-noise_0)/trad_len  + extra
    
    return(col)

In [3]:
def add_bbpoly_noise(time,trad_len,sigma):
    """A function to add the poly approx bronwian bridge 
    noise given the trajectory length and sigma"""
    #find the normal constants and renormalise by the needed k factor
    I_k = np.random.randn(5)
    for i in range(1,6):
        I_k[i-1] = I_k[i-1] * (i*(i+1))**(-1)
    
    #apply the dot function 
    noise = np.apply_along_axis(dot,0,time,I_k=I_k,trad_len=trad_len)    
    
    #quick normalisation to get it to start an end at zero    
    matrix = pd.DataFrame(np.concatenate((np.array([noise]),np.array([time])),axis = 0))
    matrix = matrix.apply(renorm, axis = 0,noise_0=noise[0],noise_1=noise[-1],trad_len=trad_len)
    noise = sigma * matrix.T[0]
    return(noise)




def add_bbpoly_noise_ep(time,trad_len,sigma,end_point):
    """A function to add the poly approx bronwian bridge 
    noise given the trajectory length and sigma to end at
    a specific end point"""
    #find the normal constants and renormalise by the needed k factor
    I_k = np.random.randn(5)
    for i in range(1,6):
        I_k[i-1] = I_k[i-1] * (i*(i+1))**(-1)
    
    #apply the dot function 
    noise = np.apply_along_axis(dot,0,time,I_k=I_k,trad_len=trad_len)    
    
    #quick normalisation to get it to start an end at zero    
    matrix = pd.DataFrame(np.concatenate((np.array([noise]),np.array([time])),axis = 0))
    matrix = matrix.apply(renorm_ep, axis = 0,noise_0=noise[0],noise_1=noise[-1],trad_len=trad_len,end_point=end_point)
    noise = sigma * matrix.T[0]
    return(noise)




def add_bbpoly_noise_sp(time,trad_len,sigma,start_point):
    """A function to add the poly approx bronwian bridge 
    noise given the trajectory length and sigma to start 
    at a specific start point"""
    #find the normal constants and renormalise by the needed k factor
    I_k = np.random.randn(5)
    for i in range(1,6):
        I_k[i-1] = I_k[i-1] * (i*(i+1))**(-1)
    
    #apply the dot function 
    noise = np.apply_along_axis(dot,0,time,I_k=I_k,trad_len=trad_len)    
    
    #quick normalisation to get it to start an end at zero    
    matrix = pd.DataFrame(np.concatenate((np.array([noise]),np.array([time])),axis = 0))
    matrix = matrix.apply(renorm_sp, axis = 0,noise_0=noise[0],noise_1=noise[-1],trad_len=trad_len,start_point=start_point)
    noise = sigma * matrix.T[0]
    return(noise)




def tight_point_noise(time,p,tight_noise,sigma,trad_len):
    #p is the tight point percentage
    plen = trad_len * p
    
    time_b1 = time[time<plen]
    time_b2_vec = time[time>=plen] 
    time_b2 = time_b2_vec - time_b2_vec.iloc[0]
        
    #simulate the tight point noise
    point =  tight_noise * np.random.randn(1)
    
    #simulate a bridge from 0 to the point
    trad_len_b1 = time_b1.iloc[-1]
    noise_b1 = add_bbpoly_noise_ep(time_b1,trad_len_b1,sigma,point)
    
    #simulate a bridge from the point to 0
    trad_len_b2 = time_b2.iloc[-1]-time_b2.iloc[0]
    noise_b2 = add_bbpoly_noise_sp(time_b2,trad_len_b2,sigma,point)
    
    noise = noise_b1.append(noise_b2,ignore_index=True)
    return(noise)

In [4]:
def add_curve(column,curve):
    #start by getting the column of times
    t = column[::3]
    t = t.reset_index(drop=True)
    
    #find the x and y values
    x = curve().x(t)
    y = curve().y(t)
    
    #add the xs and ys
    column = column[2:].reset_index(drop=True)
    column[::3]=y
    zero = pd.Series([0])
    column = pd.concat([zero, column], ignore_index=True)
    column[::3]=x
    initial_time = pd.Series([t[0]])
    column = pd.concat([initial_time, column], ignore_index=True)
    
    return(column)


def add_time(row,trad_len, acc_noise = 0.2, dt=0.1, x0 = [0,1]):
    n = np.shape(row)[0]
    
    F = np.array([[1.0, dt], [0.0, 1.0]]) #the state matrix
    sqrtU = acc_noise * np.array([[dt**2/2],[dt]]) #noise matrix
    state_noise_seq = np.random.randn(sqrtU.shape[1], n-1) #the noise sequence
    
    
    x = np.zeros((2, n))
    x[:, 0] = x0
    for k in range(1,n):
        x[:, k] = np.matmul(F, x[:, k-1]) + np.matmul(sqrtU, state_noise_seq[:, k-1])
    row = pd.Series(x[0])
    
    #renormalise times
    max_t = max(row)
    row = row * (max_t)**(-1) * trad_len
    
    #we also want to the boat to stay fnished once it reaches the end of its path
    end_point = np.argmax(row)
    if end_point!= (n-1) :
        row[end_point:] = trad_len
        
        
    #we also want the boat to not be in negative time
    row = np.where(row <= 0, 0, row) 
    row = pd.Series(row)

    return(row)



def set_up(boat_rate,time):
    max_boat_num_sim = int(boat_rate*time*2.5)
    boat_start_times = np.cumsum(np.random.exponential(1/boat_rate,max_boat_num_sim))
    boat_start_times = boat_start_times[boat_start_times<time]
    boat_number = len(boat_start_times)
    return(boat_start_times,boat_number)
 
    
    
    
def boats(curve,boat_rate,time,n,sigma,noise):
    
    boat_start_times,boat_number = set_up(boat_rate,time)
    #set up empty holding matrix
    df = pd.DataFrame(np.zeros((boat_number*3,n)))
    
    #add the times to every third row
    df_times_initial = pd.DataFrame(np.zeros((boat_number,n)))
    df_times_initial = df_times_initial.apply(add_time,trad_len=curve().trad_len,axis=1)
    df_times_initial = df_times_initial.set_index(df.iloc[::3].index)    
    df.iloc[::3] = df_times_initial

    #add the curve
    df = df.apply(add_curve,axis=0,curve=curve) #we know this works but we want to make sure we're targetting the correct row
    
    #create the noise
    time_matrix = df[::3].reset_index(drop=True)
    if noise == 'bbpa':
        x_noise = time_matrix.apply(add_bbpoly_noise,trad_len=curve().trad_len,sigma=sigma,axis=1)
        y_noise = time_matrix.apply(add_bbpoly_noise,trad_len=curve().trad_len,sigma=sigma,axis=1)
    elif noise == 'tpn':
        x_noise = time_matrix.apply(tight_point_noise,p = curve().p,tight_noise = 0.01,sigma=sigma,trad_len=curve().trad_len,axis=1)
        y_noise = time_matrix.apply(tight_point_noise,p = curve().p,tight_noise = 0.01,sigma=sigma,trad_len=curve().trad_len,axis=1)
    else:
        print('Error noise type not defined')
        
    #add the noise
    index_x = pd.RangeIndex(start=1,stop=3*boat_number,step=3)
    x_noise = x_noise.set_index(index_x,drop=True)
    df.iloc[index_x] = df.iloc[index_x].add(x_noise, fill_value=0)
    index_y = pd.RangeIndex(start=2,stop=3*boat_number,step=3)
    y_noise = y_noise.set_index(index_y,drop=True)
    df.iloc[index_y] = df.iloc[index_y].add(y_noise, fill_value=0)
    
    
    #now we need to edit the times so that they represent the start times
    start_times_series = pd.Series(boat_start_times)
    df_times = pd.concat([start_times_series]*n, axis = 1,ignore_index=True)
    df_times = df_times.set_index(df.iloc[::3].index)  
    df.iloc[::3] = df.iloc[::3].add(df_times, fill_value=0)
    
    return(df)

## Adding the polygon

In [5]:
coords = np.array([[0.3133819 , 0.81372773],
       [0.37772741, 0.95921838],
       [0.52317508, 0.85764943],
       [0.46553223, 0.77255113],
       [0.52451561, 0.62843303],
       [0.67666594, 0.73274708],
       [0.94008038, 0.370393  ],
       [0.95750729, 0.22764746],
       [0.87372407, 0.1       ],
       [0.73497906, 0.11509809],
       [0.59288272, 0.25921619],
       [0.49837525, 0.26333385],
       [0.39046246, 0.12882362],
       [0.24836612, 0.16313745],
       [0.19474486, 0.34019683],
       [0.24233373, 0.47607961],
       [0.27115515, 0.67098218],
       [0.30600897, 0.71215878],
       [0.3133819 , 0.81372773]])

In [6]:
class black_sea_obj():
    def __init__(self, borders_xy):
        self.border_points = borders_xy # Setting the array as nodes over which
        # a path is defined to define continuous borders
        self.area = Polygon(self.border_points)
        self.border = self.area.bounds
        self.ports = []
        print("Finish initialising class black_sea_obj")
    
    def set_port(self, name, port_coords):
        # Method to find the closest point to the given coordinates that is on
        # the border path
        port_coords_point = Point(port_coords)
        closest_border_point, temp = nearest_points(self.area, port_coords_point)
        x, y = closest_border_point.coords.xy[0][0], closest_border_point.coords.xy[1][0] # "nearest point gives
        # an actual "Point" class as output with many other properties other than the coords
        xy = np.array([x,y])
        print("Point is not on border. Define coordinates as " + str(closest_border_point))
        self.ports.append([name, xy])

In [7]:
black_sea = black_sea_obj(coords)

#adding the ports
black_sea.set_port('Constanta', np.array([0.25,0.54]))
black_sea.set_port('Istanbul', np.array([0,0]))
black_sea.set_port('Kerch', np.array([0.68,0.73]))
black_sea.set_port('Odessa', np.array([.4,1]))
black_sea.set_port('Varna', np.array([0.1,0.4]))

#creating a dataframe of the ports
ports_df = pd.DataFrame(data = black_sea.ports, columns = ['name','location'])

Finish initialising class black_sea_obj
Point is not on border. Define coordinates as POINT (0.2517478042703394 0.5397415416279363)
Point is not on border. Define coordinates as POINT (0.24836612 0.16313745)
Point is not on border. Define coordinates as POINT (0.6791252235796421 0.7293640785253644)
Point is not on border. Define coordinates as POINT (0.37772741 0.95921838)
Point is not on border. Define coordinates as POINT (0.2030496670887143 0.3639099434775466)


In [8]:
#plot with ports
ax = plt.gcf().gca()
ax.plot(black_sea.border_points[:,0],black_sea.border_points[:,1])

shape = ports_df.shape
n = shape[0]
number_of_colors = n
color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(number_of_colors)]

for i in range(n):
    name = ports_df['name'][i]
    ax.plot(black_sea.ports[i][1][0], black_sea.ports[i][1][1],'or', c = color[i], label = name)
    
plt.legend(loc="upper right")

In [9]:
# adding the paths
class istanbul_odessa(object):
    def __init__(self):
        self.trad_len = 1.7
        self.start = 'Istanbul'
        self.s_port_vec = ports_df[ports_df['name'] == 'Istanbul']['location'].values[0]
        self.s_port = np.array([self.s_port_vec[0],self.s_port_vec[1]])
        
        self.end = 'Odessa'
        self.e_port_vec = ports_df[ports_df['name'] == 'Odessa']['location'].values[0]
        self.e_port = np.array([self.e_port_vec[0],self.e_port_vec[1]])
    
    def x(self,t):
        n = np.shape(t)[0]
        
        y_1 = self.s_port[1]
        y_2 = self.e_port[1]
        x_1 = self.s_port[0]
        x_2 = self.e_port[0]
        y = (y_2-y_1)/self.trad_len*t + y_1*np.ones(n)
        
        a = (y_1-y_2)/(x_1-x_2)
        b = y_1-(x_1*(y_1-y_2))/(x_1-x_2)
        x = (y-b*np.ones(n))/a
        return x

    def y(self,t):
        n = np.shape(t)[0]
        y_1 = self.s_port[1]
        y_2 = self.e_port[1]
        y = (y_2-y_1)/self.trad_len*t + y_1*np.ones(n)
        return y
    
    
    
class varna_odessa(object):
    def __init__(self):
        self.trad_len = 1.2
        self.start = 'Varna'
        self.s_port_vec = ports_df[ports_df['name'] == 'Varna']['location'].values[0]
        self.s_port = np.array([self.s_port_vec[0],self.s_port_vec[1]])
        
        self.end = 'Odessa'
        self.e_port_vec = ports_df[ports_df['name'] == 'Odessa']['location'].values[0]
        self.e_port = np.array([self.e_port_vec[0],self.e_port_vec[1]])
        
        self.x_1 = self.s_port[0]
        self.x_2 = self.e_port[0]
        self.y_1 = self.s_port[1]
        self.y_2 = self.e_port[1]
        
        self.x_spl = np.array([self.x_1,0.22,0.24,0.26,0.28,0.32,0.35,self.x_2,self.x_2+0.001])
        self.y_spl = np.array([self.y_1,0.33,0.31,0.3,0.31,0.41,0.66,self.y_2,self.y_2+0.001])
        self.t_spl, self.c_spl, self.k_spl = interpolate.splrep(self.x_spl, self.y_spl, s=0, k=2)
        self.spl = interpolate.BSpline(self.t_spl, self.c_spl, self.k_spl, extrapolate=False)
    
    def x(self,t):
        x = ((self.x_2-self.x_1)/self.trad_len)*t+self.x_1
        return x

    def y(self,t):
        x = ((self.x_2-self.x_1)/self.trad_len)*t+self.x_1
        y = self.spl(x)
        return y
    
    
    
    
class constanta_kerch(object):
    def __init__(self):
        self.trad_len = 2
        self.p = 0.8
        
        self.start = 'Constanta'
        self.s_port_vec = ports_df[ports_df['name'] == 'Constanta']['location'].values[0]
        self.s_port = np.array([self.s_port_vec[0],self.s_port_vec[1]])
        
        self.end = 'Kerch'
        self.e_port_vec = ports_df[ports_df['name'] == 'Kerch']['location'].values[0]
        self.e_port = np.array([self.e_port_vec[0],self.e_port_vec[1]])
        
        self.x_1 = self.s_port[0]
        self.x_2 = self.e_port[0]
        self.y_1 = self.s_port[1]
        self.y_2 = self.e_port[1]
        
        self.x_spl = np.array([self.x_1,0.31,0.44,0.57,self.x_2])
        self.y_spl = np.array([self.y_1,0.52,0.52,0.59,self.y_2])
        self.t_spl, self.c_spl, self.k_spl = interpolate.splrep(self.x_spl, self.y_spl, s=0, k=4)
        self.spl = interpolate.BSpline(self.t_spl, self.c_spl, self.k_spl, extrapolate=False)
    
    def x(self,t):
        x = ((self.x_2-self.x_1)/self.trad_len)*t+self.x_1
        return x

    def y(self,t):
        x = ((self.x_2-self.x_1)/self.trad_len)*t+self.x_1
        y = self.spl(x)
        return y

### Plotting the paths on the polygon with the ports

In [10]:
ax = plt.gcf().gca()
ax.plot(black_sea.border_points[:,0],black_sea.border_points[:,1],color='grey')

shape = ports_df.shape
n = shape[0]
#number_of_colors = n

color = plt.cm.rainbow(np.linspace(0, 1, n))
#color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             #for i in range(number_of_colors)]

for i in range(n):
    name = ports_df['name'][i]
    ax.plot(black_sea.ports[i][1][0], black_sea.ports[i][1][1],'or', c = color[i], label = name)
    


#varna odessa
time_vo = np.arange(0,1.21,0.01)
x_vo = varna_odessa().x(time_vo)
y_vo = varna_odessa().y(time_vo)
ax.plot(x_vo,y_vo,color = 'grey')
#istanbul odessa
time_io = np.arange(0,1.71,0.01)
x_io = istanbul_odessa().x(time_io)
y_io = istanbul_odessa().y(time_io)
ax.plot(x_io,y_io,color = 'grey')
#constanta kerch
time_ck = np.arange(0,2.01,0.01)
x_ck = constanta_kerch().x(time_ck)
y_ck = constanta_kerch().y(time_ck)
ax.plot(x_ck,y_ck,color = 'grey')

plt.legend(loc="upper right")

### Create the Data

In [11]:
boats_vo = boats(varna_odessa,10,1,1000,0.1,'bbpa')

In [12]:
#Plot

ax = plt.gcf().gca()
ax.plot(black_sea.border_points[:,0],black_sea.border_points[:,1],color='grey')

shape = ports_df.shape
n = shape[0]

color = plt.cm.rainbow(np.linspace(0, 1, n))

for i in range(n):
    name = ports_df['name'][i]
    ax.plot(black_sea.ports[i][1][0], black_sea.ports[i][1][1],'or', c = color[i], label = name)
  
boat_number_vo = int(boats_vo.shape[0]/3)
for i in range(int(boat_number_vo)):
    x_boat = boats_vo.iloc[3*i+1]
    y_boat = boats_vo.iloc[3*i+2]
    ax.plot(x_boat,y_boat,color = 'red',alpha=0.5)

time_vo = np.arange(0,1.21,0.01)
x_vo = varna_odessa().x(time_vo)
y_vo = varna_odessa().y(time_vo)
ax.plot(x_vo,y_vo,color = 'grey')

plt.legend(loc="upper right")

## Plot of all major routes of interest so far

In [13]:
#Data
boats_vo = boats(varna_odessa,10,1,1000,0.1,'bbpa')
boats_ck = boats(constanta_kerch,10,1,1000,0.1,'tpn')
boats_io = boats(istanbul_odessa,10,1,1000,0.1,'bbpa')

In [15]:
#Plot

img = plt.imread("mernoire12.gif") #download background image

fig, ax = plt.subplots() #set up axes
ax.imshow(img, extent=[0, 4, 0, 3]) #add the background

#add the cities
ax.plot(0.51,1.1,'*',label='Varna')
ax.plot(1.24,2.34,'*',color = 'green',label='Odessa')
ax.plot(0.65,1.3,'*',color = 'black',label='Constanta')
ax.plot(0.89,0.72,'*',color = 'red',label='Istanbul')
ax.plot(2.51,1.91,'*',color = 'orange',label='Kerch')

#add paths
#varna odessa
time_vo = np.arange(0,1.21,0.01)
x_vo = varna_odessa().x(time_vo)
y_vo = varna_odessa().y(time_vo)
ax.plot(x_vo,y_vo,color = 'grey')

time_ck = np.arange(0,2.01,0.01)
x_ck = constanta_kerch().x(time_ck)
y_ck = constanta_kerch().y(time_ck)
ax.plot(x_ck,y_ck,color = 'grey')

time_io = np.arange(0,1.71,0.01)
x_io = istanbul_odessa().x(time_io)
y_io = istanbul_odessa().y(time_io)
ax.plot(x_io,y_io,color = 'grey')

#add boats for varna odessa
boat_number_vo = int(boats_vo.shape[0]/3)

for i in range(int(boat_number_vo)):
    x_boat = boats_vo.iloc[3*i+1]
    y_boat = boats_vo.iloc[3*i+2]
    ax.plot(x_boat,y_boat,color = 'red',alpha=0.5)

#add boats for Constanta kerch
boat_number_ck = int(boats_ck.shape[0]/3)

for i in range(int(boat_number_ck)):
    x_boat = boats_ck.iloc[3*i+1]
    y_boat = boats_ck.iloc[3*i+2]
    y_boat_len = len(y_boat[y_boat<1.3])
    ax.plot(x_boat[y_boat_len:1000],y_boat[y_boat_len:1000],color = 'red',alpha=0.5)

    
#add boats for Istanbul Odessa
boat_number_io = int(boats_io.shape[0]/3)

for i in range(int(boat_number_io)):
    x_boat = boats_io.iloc[3*i+1]
    y_boat = boats_io.iloc[3*i+2]
    ax.plot(x_boat,y_boat,color = 'red',alpha=0.5)


#adding legends
ax.legend()
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
           ncol=3, mode="expand", borderaxespad=0.)

FileNotFoundError: [Errno 2] No such file or directory: 'mernoire12.gif'

# Generating Data

To generate the data we use the function 'boats'. We need six inputs for this to run. 

* **curve**: This is the path being simulated, all of the ones given are classes above, input as a class

* **boat_rate**: This is the number of boats expected per unit of time, input as an integer

* **time**: This is how long we are simulating for, input as a float

* **n**: This is the discretisation of the path (typical use is 100), input as an integer

* **sigma**: This is the noise on the path, input as a float

* **noise**: This is the type of noise, choose one of 'bbpa' and 'tpn', should be input as strings


In [56]:
#example

boats_vo = boats(varna_odessa,1,2,1000,0.05,'bbpa')

#this gives boats between varna and odessa at a rate of ten per unit of time over one unit of time, with 1000 readings, a noise of 0.1 and brownian poly type noise

In [57]:
boats_vo

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.203866,1.205586,1.207285,1.208962,1.210689,1.212467,1.214269,1.216072,1.217900,1.219794,...,2.396828,2.397668,2.398486,2.399267,2.400033,2.400781,2.401534,2.402310,2.403084,2.403866
1,0.203050,0.203348,0.203641,0.203931,0.204228,0.204534,0.204842,0.205151,0.205462,0.205785,...,0.376772,0.376886,0.376997,0.377103,0.377207,0.377308,0.377411,0.377516,0.377621,0.377727
2,0.363910,0.363274,0.362650,0.362039,0.361415,0.360778,0.360138,0.359502,0.358864,0.358209,...,0.957543,0.957798,0.958032,0.958242,0.958436,0.958613,0.958779,0.958938,0.959084,0.959218


In [129]:
Traj = np.zeros((2,1000))
Traj[0] = boats_vo.T[1]
Traj[1] = boats_vo.T[2]

Time = boats_vo.T[0]

In [130]:
(a,N) = np.shape(boats_vo)

In [131]:
#Import numpy so that we can actually do maths ...
import numpy as np



#We define the measurement function to compute (in absence of noise) the measured values.
#This particular measurement function is based on a spherical sound-amplitude attenuation model.
def Measure(Sensors, Prediction):
    r = np.linalg.norm(Sensors.T - ([(Prediction.T[0])[0],(Prediction.T[0])[1]]) , axis = 1)
    Mea = rho - (5/np.log(10))*np.log(0.0001 + r*r)
    #If the 'measured' value would be no greater than the background noise (set to be 0),
    # the measurement function returns zero. 
    Mea = np.array([Mea]).T
    return(Mea)



#We also define a function to compute the Jacobian of the measurement transformation.
def Jacobian(Sensors , Prediction):
    Jac = np.zeros([p,4])
    for j in range(0,p):
        r = np.linalg.norm(np.array([(Prediction.T[0])[0],(Prediction.T[0])[1]]) - Sensors.T[j])
        Jac[j,0] = (-10/np.log(10))*(Prediction[0] - (Sensors.T[j])[0])/(0.0001 + r*r)
        Jac[j,1] = (-10/np.log(10))*(Prediction[1] - (Sensors.T[j])[1])/(0.0001 + r*r)
    return(Jac)





# Measurements simulation ##################################################################################
# Uses the simulated trajectory to produce simulated measurements to be passed to the EKF
# - S - Sensor positions
S = np.array([[0.3,0.375,0.2,0.35,0.4],[0.6,1,0.4,0.4,0.9]])
# - M/R - Measurement error and covariance matrix
M = 0.05
#####################################################################
# The value of M should be set/adjusted depending on what a         #
# representative measurment variance is thought to be.              #
#####################################################################
R = M*M*np.eye(5)
# - rho - ?Range parameter?
rho = -5#
#####################################################################
# The value of rho is not very interpretable, now that we've        #
# non-dimensionalised using the length of the black sea; this will  #
# need to be set/adjusted depending on what a representative sensor #
# range is thought to be.                                           #
#####################################################################

Measu = np.zeros((5,N))
for i in range(0,N):
    Measu.T[i] = np.array([np.maximum( 0 , Measure(S, np.array([Traj.T[i]]).T).T + np.array([np.random.multivariate_normal(np.zeros(5),R)]))])






#Extended Kalman Filter for single-target tracking:

#Inputs:
# - X0 - an initial estimate for the state vector.
# - P0 - an estimated variance/uncertainty of the initial estimate.
# - Measu - the simulated measurements generated above
# - S - a 2-by-p matrix with the sensors' x-coordinates in the first row, and y-coordinates in the second

def EKF(X0, P0, Measu, S, q1, q2):
    p = np.shape(Measu)[0]
    #Parameters:
    # - cutoff - a number of consecutive ''no measurement''s after which the filter will stop
    cutoff = 5
    
    # - A - the state-state transition matrix; this A assumes a linear (approximation to the) evolution equation:
    A = np.eye(4)
    # - this will change with each iterate; these are simply the constant elements in A
    
    #Initialise arrays to record the estimated and 'measured' values.
    Estim = np.zeros((4,N))
    Trace = np.zeros((1,N))
    Sigma = np.zeros((1,N))
    Y = np.zeros((p,1))
    
    #Start an iteration counter.
    i = 0
    
    #Write the initial estimate into the record.
    Estim.T[0] = X0.T
    Trace.T[0] = np.trace(P0)
    Sigma.T[0] = P0[0,0] + P0[1,1]
        
    #Start the filter
    
    #X -- State matrix
    X = X0
    
    #P -- State covariance matrix
    P = P0
    
    zerocount = 0
    
    while (i < N-1)&(zerocount < cutoff):
        # - Dt - the time interval between iterations
        Dt = Time[i+1] - Time[i]
        
        A[0,2] = Dt
        A[1,3] = Dt
        
        #Xp -- Predicted state
        Xp = A@X                        
        
        #Pp -- Predicted state covariance
        #Q -- (model) process noise covariance
        Q = q1*np.array([[Dt*Dt*Dt*Dt/4, 0, Dt*Dt*Dt/2, 0],
                     [0, 0, 0, 0],
                     [Dt*Dt*Dt/2, 0, Dt*Dt, 0],
                     [0, 0, 0, 0]]) + q2*np.array([[0, 0, 0, 0],
                     [0, Dt*Dt*Dt*Dt/4, 0, Dt*Dt*Dt/2],
                     [0, 0, 0, 0],
                     [0, Dt*Dt*Dt/2, 0, Dt*Dt]])
        Pp = A@P@A.T + Q
        ########################################################################
        # The process noise matrix parameters (q1,q2) act as tuning parameters #
        ########################################################################
        
        #Y -- 'Measured' value
        Y = Measu.T[i].T
        if np.linalg.norm(Y) == 0:
            zerocount = zerocount + 1
        else:
            zerocount = 0    
        
        H = (Jacobian(S,Xp).T * (1*(Y > 0))).T
        # We zero out the Jacobian entries corresponding to zero measurements
        
        #K -- Kalman gain
        K = Pp@H.T@np.linalg.inv(H@Pp@H.T + R)
        
        #Update X and P
        X = Xp + K@((Y - Measure(S,Xp).T).T)
        P = (np.eye(4) - K@H)@Pp
        
        #Increment the iteration counter
        i = i+1
        
        #Write the new values to the record.
        Estim.T[i] = X.T
        Trace.T[i] = np.trace(P)
        Sigma.T[i] = P[0,0] + P[1,1]
    
    
    # Remove extraneous record entries:
    if zerocount == cutoff:
        n = i - cutoff
        Measu = np.delete(Measu,np.arange(N-n)+n,1) # This is now a p-by-n array
        Estim = np.delete(Estim,np.arange(N-n)+n,1) # This is now a 4-by-n array
        Trace = np.delete(Trace,np.arange(N-n)+n,1) # This is now a 1-by-n array
        Sigma = np.delete(Sigma,np.arange(N-n)+n,1) # This is now a 1-by-n array
    else:
        n = N
    
    return(n, Measu, Estim, Trace, Sigma)

# Outputs:
# - n - the number of iterations performed
# - Measu - the sound measurements from the p sensors, stored as the entries of a p-by-n array
# - Estim - the sequential state-vector estimates, stored as the columns of a 4-by-n array
# - Trace - the traces of the sequential state covariance matrices, stored as the entries of a 1-by-n array
# - Sigma - the sum of the position variances (the first two diagonal entries of the state covariance matrices),
#            stored as the entries of a 1-by-n array

In [132]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.plot(Traj[0], Traj[1],'k',label='Trajectory')
ax1.plot(S[0,0],S[1,0],'o',label='Sensor 1')
ax1.plot(S[0,1],S[1,1],'o',label='Sensor 2')
ax1.plot(S[0,2],S[1,2],'o',label='Sensor 3')
ax1.plot(S[0,3],S[1,3],'o',label='Sensor 4')
ax1.plot(S[0,4],S[1,4],'o',label='Sensor 5')
ax1.set_xlabel('x-displacement', fontsize=16)
ax1.set_ylabel('y-displacement', fontsize=16)
ax1.legend()

xt = (np.arange(1000))
ax2.plot(xt, Measu[0],label='Sensor 1')
ax2.plot(xt, Measu[1],label='Sensor 2')
ax2.plot(xt, Measu[2],label='Sensor 3')
ax2.plot(xt, Measu[3],label='Sensor 4')
ax2.plot(xt, Measu[4],label='Sensor 5')
ax2.set_xlabel('iterate', fontsize=16)
ax2.set_ylabel('measurement', fontsize=16)
ax2.legend()

In [133]:
X0 = np.array([[0.20304967], [0.36390994],[0],[0]])
P0 = np.diag([0.000001,0.000001,0.000001,0.000001])
p = 5
(n, Measu, Estim, Trace, Sigma) = EKF(X0, P0, Measu, S, 25, 15)

##########################################################
# X0, P0, q1, and q2 can be considered tuning parameters #
##########################################################

In [134]:
fig, axs = plt.subplots(2, 2, figsize=(12, 12))

axs[0,0].plot(Traj[0],Traj[1],'k',label='Trajectory')
axs[0,0].plot(Estim[0],Estim[1],'b',label='Estimates')
axs[0,0].plot(S[0,0],S[1,0],'o',label='Sensor 1')
axs[0,0].plot(S[0,1],S[1,1],'o',label='Sensor 2')
axs[0,0].plot(S[0,2],S[1,2],'o',label='Sensor 3')
axs[0,0].plot(S[0,3],S[1,3],'o',label='Sensor 4')
axs[0,0].plot(S[0,4],S[1,4],'o',label='Sensor 5')
axs[0,0].set_xlabel('x-displacement', fontsize=16)
axs[0,0].set_ylabel('y-displacement', fontsize=16)
axs[0,0].legend()

xt = (np.arange(n))
#axs[0,1].plot(xt,Trace[0],label='Trace')
axs[0,1].plot(xt,Sigma[0])
axs[0,1].set_ylabel('Position uncertainty',fontsize=16)
axs[0,1].set_xlabel('iterate',fontsize=16)

SquError = (Traj[0] - Estim[0])*(Traj[0] - Estim[0]) + (Traj[1] - Estim[1])*(Traj[1] - Estim[1])
RMSE = np.sqrt(np.sum(SquError)/n)
axs[1,0].plot(xt,np.sqrt(SquError))
axs[1,0].plot(xt,RMSE + xt*0,label='RMSE')
axs[1,0].set_ylabel('(Absolute) position error', fontsize=14)
axs[1,0].set_xlabel('iterate', fontsize=14)
axs[1,0].legend()

RSE = np.cumsum(np.sqrt(SquError))
axs[1,1].plot(xt,RSE)
axs[1,1].set_ylabel('Cumulative absolute position error', fontsize=14)
axs[1,1].set_xlabel('iterate', fontsize=14)

Text(0.5, 0, 'iterate')